In [1]:
import string
import os
import sys
import subprocess
import shutil
import time
import webbrowser
import numpy as np

from glob import glob
from pathlib import Path, PosixPath
from IPython.display import HTML, IFrame, Image, display, clear_output

# Neuroimaging Modules
import IPython
import pydicom
import nibabel as nib
import dipy.reconst.dti as dti
from dipy.segment.mask import median_otsu
from dipy.denoise.nlmeans import nlmeans
from dipy.denoise.noise_estimate import estimate_sigma
from dipy.core.gradients import gradient_table
from dipy.io import read_bvals_bvecs
from dipy.reconst.dti import fractional_anisotropy
from dipy.io.bvectxt import reorient_vectors

from lib.Utils.PNGViewer.PNGViewer import PNGViewer

In [2]:
def mrtrix_preproc(dwi_pre, eddy_options):
    #1. Convert to MIF 
    !mrconvert -force -nthreads 12 -fslgrad $input_bvec $input_bval $input_dwi $dwi_raw
    #2. Denoise
    !dwidenoise -force -nthreads 12 $dwi_raw $dwi_den
    #3. Remove Gibbs rings
    !mrdegibbs -force -nthreads 12 $dwi_den $dwi_den_deg
    #4. Preprocess
    !dwifslpreproc -force -info -nthreads 12 $dwi_den_deg $dwi_den_deg_preproc -pe_dir AP -rpe_none -scratch '/tmp' -eddy_options $eddy_options -eddyqc_all $preproc_out_eddyqc
    #5. Bias correction
    !dwibiascorrect -force -nthreads 12 -ants $dwi_den_deg_preproc $dwi_den_deg_preproc_unb -scratch '/tmp'
    #6. Mask generation
    ##a. Extract b0s
    !dwiextract -force -nthreads 12 -bzero $dwi_den_deg_preproc_unb $dwi_den_deg_preproc_unb_b0s
    ##b. Compute mean b0
    !mrmath -force -nthreads 12 -axis 3 $dwi_den_deg_preproc_unb_b0s mean $dwi_den_deg_preproc_unb_meanb0
    ##c. Convert mean b0 to NII
    !mrconvert -force -nthreads 12 $dwi_den_deg_preproc_unb_meanb0 $dwi_den_deg_preproc_unb_meanb0NII
    ##d. Create mask
    !bet2 $dwi_den_deg_preproc_unb_meanb0NII $dwi_den_deg_preproc_unb_meanb0maskroot -m
    ##e. Convert mask back to MIF
    !mrconvert -force -nthreads 12 $dwi_den_deg_preproc_unb_meanb0maskNIIGZ $dwi_den_deg_preproc_unb_meanb0mask

In [ ]:
BIDSmasterdir = Path("/Volumes/Vol6/YouthPTSD/BIDS_master")
BIDSprocdir = Path("/Volumes/Vol6/YouthPTSD/BIDS_Processed")
Scratchprocdir = Path("/scratch/jdrussell3/dwiproc")

for subjdir in sorted(BIDSmasterdir.glob('sub*')):
    for sesdir in sorted(subjdir.glob('ses*')):
        for dwidir in sesdir.glob('dwi'):
            output_dir = Path(Scratchprocdir, subjdir.name, sesdir.name, 'dwi')
            if output_dir.exists():
                shutil.rmtree(output_dir)
            output_dir.mkdir(parents=True)
            for rawdwi in dwidir.glob('*.nii*'):
                #Get sub-XXX_ses-YY string (dwi_pre)
                baseout_list = rawdwi.name.split('_')[0:2]
                s = "_"
                baseout_stem = s.join(baseout_list)
                dwi_pre = baseout_stem
                
                #Copy original files from BIDS_Master to BIDS_Processesd/../../original
                orig_dwi = rawdwi
                orig_bval = rawdwi.with_suffix('.bval')
                orig_bvec = rawdwi.with_suffix('.bvec')
                
                new_orig_dir = Path(output_dir, 'original')
                new_orig_dir.mkdir(parents=True)
                new_orig_dwi = new_orig_dir / str(dwi_pre + "_dwi.nii")
                new_orig_bval = new_orig_dir / str(dwi_pre + "_dwi.bval")
                new_orig_bvec = new_orig_dir / str(dwi_pre + "_dwi.bvec")
                shutil.copyfile(orig_dwi, new_orig_dwi)
                shutil.copyfile(orig_bval, new_orig_bval)
                shutil.copyfile(orig_bvec, new_orig_bvec)
                
                #Begin defining preprocessing variables
                preproc_out = Path(output_dir, "preproc_out")
                preproc_out.mkdir(parents=True)
                preproc_out_eddyqc = Path(preproc_out, "eddy_qc")
                temp_dir = '/tmp'
                eddy_options = "'--verbose --slm=linear '"
                input_dwi = new_orig_dwi
                input_bvec = new_orig_bvec
                input_bval = new_orig_bval
                dwi_raw = Path(preproc_out, str(dwi_pre + "_dwi.mif"))
                dwi_den = Path(preproc_out, str(dwi_pre + "_dwi_den.mif"))
                dwi_den_deg = Path(preproc_out, str(dwi_pre + "_dwi_den_deg.mif"))
                dwi_den_deg_preproc = Path(preproc_out, str(
                    dwi_pre + "_dwi_den_deg_preproc.mif"))
                dwi_den_deg_preproc_unb = Path(preproc_out, str(
                    dwi_pre + "_dwi_den_deg_preproc_unb.mif"))
                dwi_den_deg_preproc_unb_b0s = Path(preproc_out, str(
                    dwi_pre + "_dwi_den_deg_preproc_unb_b0s.mif"))
                dwi_den_deg_preproc_unb_meanb0 = Path(preproc_out, str(
                    dwi_pre + "_dwi_den_deg_preproc_unb_meanb0.mif"))
                dwi_den_deg_preproc_unb_meanb0NII = Path(preproc_out, str(
                    dwi_pre + "_dwi_den_deg_preproc_unb_meanb0.nii"))
                dwi_den_deg_preproc_unb_meanb0maskroot = Path(
                    preproc_out, str(dwi_pre + "_dwi_den_deg_preproc_unb_meanb0"))
                dwi_den_deg_preproc_unb_meanb0maskNIIGZ = Path(preproc_out, str(
                    dwi_pre + "_dwi_den_deg_preproc_unb_meanb0_mask.nii.gz"))
                dwi_den_deg_preproc_unb_meanb0mask = Path(preproc_out, str(
                    dwi_pre + "_dwi_den_deg_preproc_unb_meanb0_mask.mif"))
                dwi_den_deg_preproc_unb_meanb0mask_templ = Path(preproc_out, str(
                    dwi_pre + "_dwi_den_deg_preproc_unb_meanb0_mask_templ.mif"))
                dwi_den_deg_preproc_unb_meanb0mask_ups = Path(preproc_out, str(
                    dwi_pre + "_dwi_den_deg_preproc_unb_meanb0_mask_ups.mif"))
                
                mrtrix_preproc(dwi_pre, eddy_options)
                #group_average_response = Path(preproc_out, "sub-023_ses-01_response.txt")
                #output_fod = Path(preproc_out, "sub-023_ses-01_fod.mif")
                #mask = "/scratch/jdrussell3/dipytest/meanb0_brain_mask.nii.gz"
                #mask_template = "/scratch/jdrussell/"
                
                
                #mrtrix_preproc(dwi_pre, eddy_options)

mrconvert: [WARNING] existing output files will be overwritten
mrconvert: [100%] copying from "/scratch/j...inal/sub-001_ses-01_dwi.nii" to "/scratch/j..._out/sub-001_ses-01_dwi.mif"
dwidenoise: [WARNING] existing output files will be overwritten
dwidenoise: [100%] preloading data for "/scratch/jdrussell3/dwiproc/sub-001/ses-01/dwi/preproc_out/sub-001_ses-01_dwi.mif"
dwidenoise: [100%] running MP-PCA denoising
mrdegibbs: [WARNING] existing output files will be overwritten
mrdegibbs: [100%] performing Gibbs ringing removal
dwifslpreproc: 
dwifslpreproc: Note that this script makes use of commands / algorithms that have relevant articles for citation; INCLUDING FROM EXTERNAL SOFTWARE PACKAGES. Please consult the help page (-help option) for more information.
dwifslpreproc: 
dwifslpreproc: Loading header for image file '/scratch/jdrussell3/dwiproc/sub-001/ses-01/dwi/preproc_out/sub-001_ses-01_dwi_den_deg.mif'
dwifslpreproc: Generated scratch directory: /tmp/dwifslpreproc-tmp-CCPRMM/
Comma

Command:  mrconvert result.mif /scratch/jdrussell3/dwiproc/sub-001/ses-01/dwi/preproc_out/sub-001_ses-01_dwi_den_deg_preproc.mif
          mrconvert: [WARNING] existing output files will be overwritten
          mrconvert: [100%] copying from "result.mif" to "/scratch/j...-01_dwi_den_deg_preproc.mif"
dwifslpreproc: Changing back to original directory (/Volumes/Users/jdrussell3/NeuroScripts/Jupyter)
dwifslpreproc: Deleting scratch directory (/tmp/dwifslpreproc-tmp-CCPRMM/)

Error: argument algorithm: invalid choice: '/scratch/jdrussell3/dwiproc/sub-001/ses-01/dwi/preproc_out/sub-001_ses-01_dwi_den_deg_preproc.mif' (choose from 'ants', 'fsl')
Usage: dwibiascorrect algorithm [ options ] ...
       (Run dwibiascorrect -help for more information)

dwiextract: [WARNING] existing output files will be overwritten
dwiextract: [ERROR] failed to open key/value file "/scratch/jdrussell3/dwiproc/sub-001/ses-01/dwi/preproc_out/sub-001_ses-01_dwi_den_deg_preproc_unb.mif": No such file or directory
dw

dwiextract: [WARNING] existing output files will be overwritten
dwiextract: [ERROR] failed to open key/value file "/scratch/jdrussell3/dwiproc/sub-003/ses-01/dwi/preproc_out/sub-003_ses-01_dwi_den_deg_preproc_unb.mif": No such file or directory
dwiextract: [ERROR] error opening image "/scratch/jdrussell3/dwiproc/sub-003/ses-01/dwi/preproc_out/sub-003_ses-01_dwi_den_deg_preproc_unb.mif"
mrmath: [WARNING] existing output files will be overwritten
mrmath: [ERROR] failed to open key/value file "/scratch/jdrussell3/dwiproc/sub-003/ses-01/dwi/preproc_out/sub-003_ses-01_dwi_den_deg_preproc_unb_b0s.mif": No such file or directory
mrmath: [ERROR] error opening image "/scratch/jdrussell3/dwiproc/sub-003/ses-01/dwi/preproc_out/sub-003_ses-01_dwi_den_deg_preproc_unb_b0s.mif"
mrconvert: [WARNING] existing output files will be overwritten
mrconvert: [ERROR] failed to open key/value file "/scratch/jdrussell3/dwiproc/sub-003/ses-01/dwi/preproc_out/sub-003_ses-01_dwi_den_deg_preproc_unb_meanb0.mif": No